In [1]:
# Copyright (c) OpenMMLab. All rights reserved.
import os
import tempfile
import warnings
from argparse import ArgumentParser

from mmpose.apis import (inference_top_down_pose_model, init_pose_model,
                         process_mmdet_results, vis_pose_result)
from mmpose.datasets import DatasetInfo

try:
    from mmdet.apis.inference import inference_detector, init_detector
    has_mmdet = True
except (ImportError, ModuleNotFoundError):
    has_mmdet = False

from tqdm.notebook import tqdm
import cv2
import pickle
import json

/home/lea/anaconda3/envs/vitpose/lib/python3.8/site-packages/requests/__init__.py:86: RequestsDependencyWarning: Unable to find acceptable character detection dependency (chardet or charset_normalizer).
  warnings.warn(
/home/lea/anaconda3/envs/vitpose/lib/python3.8/site-packages/albumentations/__init__.py:13: UserWarning: A new version of Albumentations is available: 2.0.8 (you have 1.4.18). Upgrade using: pip install -U albumentations. To disable automatic update checks, set the environment variable NO_ALBUMENTATIONS_UPDATE to 1.
  check_for_updates()


apex is not installed
apex is not installed
apex is not installed


## Model config

In [2]:
det_config = "demo/mmdetection_cfg/faster_rcnn_r50_fpn_coco.py" 
det_checkpoint = "checkpoints/faster_rcnn_r50_fpn_1x_coco_20200130-047c8118.pth" 
pose_config = "configs/body/2d_kpt_sview_rgb_img/topdown_heatmap/coco/ViTPose_large_coco_256x192.py"  
pose_checkpoint = "checkpoints/vitpose-l.pth"
device = "cuda:0"

det_cat_id = 1 # Category id for bounding box detection model
bbox_thr = 0.5 # Bounding box score threshold
kpt_thr = 0.5 # Keypoint score threshold
radius = 4 # Keypoint radius for visualization
thickness = 1 # Link thickness for visualization

det_model = init_detector(det_config, det_checkpoint, device=device.lower())

# build the pose model from a config file and a checkpoint file
pose_model = init_pose_model(
    pose_config, pose_checkpoint, device=device.lower())

dataset = pose_model.cfg.data['test']['type']
dataset_info = pose_model.cfg.data['test'].get('dataset_info', None)
if dataset_info is None:
    warnings.warn(
        'Please set `dataset_info` in the config.'
        'Check https://github.com/open-mmlab/mmpose/pull/663 for details.',
        DeprecationWarning)
else:
    dataset_info = DatasetInfo(dataset_info)

load checkpoint from local path: checkpoints/faster_rcnn_r50_fpn_1x_coco_20200130-047c8118.pth
load checkpoint from local path: checkpoints/vitpose-l.pth


## Run model on frames

### without rotation

In [3]:
def run_no_rotate(img_name, frame_names, out_img_root):
    drawn_frames = []
    frame_paths = [os.path.join(img_name, frame_name) for frame_name in frame_names]
    for i, path in tqdm(enumerate(frame_paths)):
        out_file = os.path.join(out_img_root, f'vis_{frame_names[i]}')

        # test a single image, the resulting box is (x1, y1, x2, y2)
        mmdet_results = inference_detector(det_model, path)

        # keep the person class bounding boxes.
        person_results = process_mmdet_results(mmdet_results, det_cat_id)
        pose_results, returned_outputs = inference_top_down_pose_model(
            pose_model,
            path,
            person_results,
            bbox_thr=bbox_thr,
            format='xyxy',
            dataset=dataset,
            dataset_info=dataset_info,
            return_heatmap=False,
            outputs=None)
        
        vis_pose_result(
            pose_model,
            path,
            pose_results,
            dataset=dataset,
            dataset_info=dataset_info,
            kpt_score_thr=kpt_thr,
            radius=radius,
            thickness=thickness,
            show=False,
            out_file=out_file)  
        
        drawn_frame = cv2.imread(out_file)
        drawn_frames.append(drawn_frame)
    
    return drawn_frames

### with rotation

In [ ]:
from Pose2Sim.poseEstimation import save_to_openpose

def run_rotate(img_name, frame_names, out_img_root, keypoint_pkl_path, pose_dir):
    drawn_frames = []
    all_keypoints = {}  # frame_name → list of person dicts
    frame_paths = [os.path.join(img_name, frame_name) for frame_name in frame_names]

    for frame_idx, image_file in tqdm(enumerate(frame_paths)):
        out_file = os.path.join(out_img_root, f'vis_{frame_names[frame_idx]}')

        # rotate to portrait
        """ img = cv2.imread(image_file)
        rotated_img = cv2.rotate(img, cv2.ROTATE_90_COUNTERCLOCKWISE)

        tmp_fd, tmp_img_path = tempfile.mkstemp(suffix=".png")
        os.close(tmp_fd)
        cv2.imwrite(tmp_img_path, rotated_img) """

        mmdet_results = inference_detector(det_model, image_file)
        person_results = process_mmdet_results(mmdet_results, det_cat_id)

        pose_results, returned_outputs = inference_top_down_pose_model(
            pose_model,
            image_file,
            person_results,
            bbox_thr=bbox_thr,
            format='xyxy',
            dataset=dataset,
            dataset_info=dataset_info,
            return_heatmap=False,
            outputs=None)

        keypoints = [r['keypoints'][:, :2] for r in pose_results]
        scores = [r['keypoints'][:, 2] for r in pose_results]

        json_output_dir = os.path.join(pose_dir, f'{os.path.basename(img_name)}_json')
        json_file_path = os.path.join(json_output_dir, f"{os.path.splitext(os.path.basename(image_file))[0]}_{frame_idx:06d}.json")
        
        save_to_openpose(json_file_path, keypoints, scores)

        # Collect keypoints with person ID
        keypoints_per_frame = []
        for pid, result in enumerate(pose_results):
            keypoints_per_frame.append({
                "id": pid,
                "bbox": result['bbox'],  # [x1, y1, x2, y2]
                "keypoints": result['keypoints'].tolist()
            })

        all_keypoints[frame_names[frame_idx]] = keypoints_per_frame

        vis_pose_result(
            pose_model,
            image_file,
            pose_results,
            dataset=dataset,
            dataset_info=dataset_info,
            kpt_score_thr=kpt_thr,
            radius=radius,
            thickness=thickness,
            show=False,
            out_file=out_file)

        drawn_frame = cv2.imread(out_file)
        drawn_frames.append(drawn_frame)

        #os.remove(tmp_img_path)

    # Save all keypoints as .pkl
    with open(keypoint_pkl_path, 'wb') as f:
        pickle.dump(all_keypoints, f)

    return drawn_frames


## Convert results to video

In [5]:
def convert_to_video(drawn_frames, out_img_root, img_name):
    output_video_path = os.path.join(out_img_root, img_name.split('/')[-1].split('.')[0] + '_vit.mp4')
    height, width, _ = drawn_frames[0].shape

    fourcc = cv2.VideoWriter_fourcc(*"mp4v")
    video_writer = cv2.VideoWriter(output_video_path, fourcc, fps=120.0, frameSize=(width, height))

    for frame in drawn_frames:
        video_writer.write(frame)

    video_writer.release()

## Data config

In [12]:
path_root = "/mnt/D494C4CF94C4B4F0/Trampoline_avril2025/Images_trampo_avril2025/20250429"
out_root = path_root + "_vitL"
keypoint_path = path_root+ "_keypts"
pose_dir = path_root + "_pose"

os.makedirs(out_root, exist_ok=True)
#os.makedirs(keypoint_path, exist_ok=True)
os.makedirs(pose_dir, exist_ok=True)

files = os.listdir(path_root)
sorted_files = sorted(files)

for i in tqdm(sorted_files):
    print(i)
    if i + '_vis' not in os.listdir(out_root):
        img_name = os.path.join(path_root, i)
        out_img_root = os.path.join(out_root, i + '_vis')
        frame_names = os.listdir(img_name)
        frame_names.sort(key=lambda x: int(x.split('.')[0].split('_')[1]))
        
        drawn_frames = run_rotate(img_name, frame_names, out_img_root, keypoint_path+'/'+i+'.pkl', pose_dir)
        #convert_to_video(drawn_frames, out_img_root, img_name)

  0%|          | 0/712 [00:00<?, ?it/s]

1_partie_0429-Camera1_M11139
1_partie_0429-Camera2_M11140
1_partie_0429-Camera3_M11141
1_partie_0429-Camera4_M11458
1_partie_0429-Camera5_M11459
1_partie_0429-Camera6_M11461
1_partie_0429-Camera7_M11462
1_partie_0429-Camera8_M11463
1_partie_0429_000-Camera1_M11139
1_partie_0429_000-Camera2_M11140
1_partie_0429_000-Camera3_M11141
1_partie_0429_000-Camera4_M11458
1_partie_0429_000-Camera5_M11459
1_partie_0429_000-Camera6_M11461
1_partie_0429_000-Camera7_M11462
1_partie_0429_000-Camera8_M11463
1_partie_0429_001-Camera1_M11139
1_partie_0429_001-Camera2_M11140
1_partie_0429_001-Camera3_M11141
1_partie_0429_001-Camera4_M11458
1_partie_0429_001-Camera5_M11459
1_partie_0429_001-Camera6_M11461
1_partie_0429_001-Camera7_M11462
1_partie_0429_001-Camera8_M11463
1_partie_0429_002-Camera1_M11139
1_partie_0429_002-Camera2_M11140
1_partie_0429_002-Camera3_M11141
1_partie_0429_002-Camera4_M11458
1_partie_0429_002-Camera5_M11459
1_partie_0429_002-Camera6_M11461
1_partie_0429_002-Camera7_M11462
1_partie_

0it [00:00, ?it/s]

3_partie_0429_018-Camera7_M11462


0it [00:00, ?it/s]

...

7_partie_0429_014-Camera7_M11462


0it [00:00, ?it/s]

7_partie_0429_014-Camera8_M11463


0it [00:00, ?it/s]

7_partie_0429_015-Camera1_M11139
7_partie_0429_015-Camera2_M11140
7_partie_0429_015-Camera3_M11141
7_partie_0429_015-Camera4_M11458
7_partie_0429_015-Camera5_M11459
7_partie_0429_015-Camera6_M11461
7_partie_0429_015-Camera7_M11462
7_partie_0429_015-Camera8_M11463
7_partie_0429_016-Camera1_M11139
7_partie_0429_016-Camera2_M11140
7_partie_0429_016-Camera3_M11141
7_partie_0429_016-Camera4_M11458
7_partie_0429_016-Camera5_M11459
7_partie_0429_016-Camera6_M11461
7_partie_0429_016-Camera7_M11462
7_partie_0429_016-Camera8_M11463
